<a href="https://colab.research.google.com/github/MoLo404Ko/VGG16_classifierLocation/blob/main/VGG16_classifier_location.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import tensorflow as tf
import pandas as pd
import numpy as np
import keras
from keras import utils as np_utils
from keras.models import Sequential
from keras.layers import Flatten, Dense
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
import os
from glob import glob
from PIL import Image
from keras.applications import vgg16
from keras.applications.imagenet_utils import preprocess_input

**Загружаем изображения**

In [99]:
def loadPathsImages():
  image_dir = '/content/sample_data/images'
  img_df = pd.read_excel('/content/sample_data/dataset/dataset.xlsx')
  names = img_df['Путь']
  paths = []

  for name in names:
    full_path = image_dir + "/" + str(name)
    paths.append(full_path)
    print(full_path)

  paths = np.asarray(paths)
  return paths

In [80]:
def loadImages(paths):
  images = []
  for path in paths:
    image = Image.open(path)
    array = np.array(image)
    images.append(array)

  return images

**Уменьшаем размер до 224x224 и нормализуем**

In [54]:
def resizeImages(images):
  reImages = []

  for img in images:
    new_img = cv2.resize(img, (224, 224))
    new_img = new_img / 255
    reImages.append(new_img)

  return reImages

**Классы изображений**

In [122]:
classes = {1:'Лес', 2:'Улица', 3:'Железная дорога', 4:'Кухня', 5:'Набережная', 6:'Комната', 7:'Пляж', 8:
 'Гора', 9:'Море', 10:'Река'}

**Загружаем метки и кодириуем по OHE**

In [100]:
def loadLabels():
  dir = '/content/sample_data/dataset/dataset.xlsx'
  df = pd.read_excel(dir)

  labels_df = df['Метка класса']
  labels = []

  for label in labels_df:
    # Метки идут от 1, поэтому - 1
    labels.append(label - 1)

  labels = np.array(labels)
  label_cat = keras.utils.to_categorical(labels)

  return label_cat

**Подготавливаем модель**

In [101]:
def prepareModel():
  vgg_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

  model = Sequential()
  model.add(vgg_model)
  model.add(Flatten())
  model.add(Dense(256, activation='relu'))
  model.add(Dense(10, activation='softmax'))

  # Разблокируем все последние слои для обучения
  vgg_model.trainable = True
  trainable = False
  for layer in vgg_model.layers:
      if layer.name == 'block5_conv1':
          trainable = True
      layer.trainable = trainable

  return model

**Компилируем и обучаем модель**

In [102]:
def trainModel(model):
  model = prepareModel()
  model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics='accuracy')
  model.fit(X_train, y_train, epochs=3, batch_size=32, verbose=1)

In [ ]:
paths = loadPathsImages()
images = loadImages(paths)

X_train = np.asarray(resizeImages(images))
y_train = loadLabels()

In [ ]:
model = prepareModel()
trainModel(model)

**Тест модели**

In [123]:
image = Image.open('/content/sample_data/test.jpg')
image = np.array(image)
image = cv2.resize(image, (224, 224))
image = image / 255

print(X_train[0].shape)
print(image.shape)
image = np.expand_dims(image, axis=0)
pred = model.predict(image)
print(classes[np.argmax(pred)])

(224, 224, 3)
(224, 224, 3)
1/1 [==============================] - 1s 555ms/step
Гора
